In [1]:
import pandas as pd
import numpy as np
from gurobipy import *

In [2]:
p = [2.5,3.5]

In [3]:
df=pd.read_csv('data/sample_disc_0.2_0.8.csv')

In [4]:
K_list = np.array(df['K']).astype(float)
L_list = np.array(df['L']).astype(float)
w_list = np.array(df['w'])

In [5]:
K_list[K_list==0]=0.001
L_list[L_list==0]=0.001

In [6]:
q1=Model('question1')
x1 = [0 for i in range(len(K_list))]
x2 = [0 for i in range(len(K_list))]
ep = [0 for i in range(len(K_list))]
y = [0 for i in range(len(K_list))]
a = 0
b = 0

for i in range(len(K_list)):
    x1[i] = q1.addVar(vtype=GRB.CONTINUOUS, name='x1'+str(i+1),lb=-np.inf)
    x2[i] = q1.addVar(vtype=GRB.CONTINUOUS, name='x2'+str(i+1),lb=-np.inf)
    y[i] = q1.addVar(vtype=GRB.CONTINUOUS, name='y'+str(i+1))
    ep[i] = q1.addVar(vtype=GRB.CONTINUOUS, name='ep'+str(i+1))
q1.update()
a = q1.addVar(vtype=GRB.CONTINUOUS, name='a')
b = q1.addVar(vtype=GRB.CONTINUOUS, name='b')
q1.update()

objective = 0
for i in range(len(K_list)):
    objective += ep[i]*ep[i]
      
q1.setObjective(objective, GRB.MINIMIZE)

Using license file /Users/lorraine/gurobi.lic
Academic license - for non-commercial use only


In [7]:
for i in range(len(K_list)):
    const = LinExpr()
    const += - p[0]*y[i] -x1[i]
    q1.addConstr(lhs=const,sense=GRB.LESS_EQUAL,rhs=0,name='py<=x1_'+str(i))
    const = LinExpr()
    const += - p[1]*y[i] -x2[i]
    q1.addConstr(lhs=const,sense=GRB.LESS_EQUAL,rhs=0,name='py<=x2_'+str(i))
    
    const = LinExpr()
    const += x1[i]*K_list[i] + x2[i]*L_list[i] + w_list[i] * y[i] -ep[i]
    q1.addConstr(lhs=const,sense=GRB.LESS_EQUAL,rhs=0,name='x1*K+x2*L+w*y_'+str(i))
    
    const = LinExpr()
    const += a+b
    q1.addConstr(lhs=const,sense=GRB.EQUAL,rhs=1,name='a+b=1')
    
    for k in range(0,int(w_list[i]/2.5)+1):
        if k==0:
            k=0.001
        for l in range(0,int(w_list[i]/3.5)+1):
            if l==0:
                l=0.001
            cost = 2.5*k +3.5*l
            if (cost <= w_list[i]): 
                _k = k
                _l =l
                const = LinExpr()
                const += a*np.log(k) - a*np.log(K_list[i]) + b*np.log(l) - b*np.log(L_list[i]) - x1[i] * (k-K_list[i]) - x2[i] * (l-L_list[i])
                q1.addConstr(lhs=const,sense=GRB.LESS_EQUAL,rhs=0,name='subgrad'+str(l)+'_'+str(k))
q1.update()

In [8]:
q1.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 24201 rows, 3990 columns and 77872 nonzeros
Model fingerprint: 0xd3935089
Model has 997 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-03, 3e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4064 rows and 361 columns
Presolve time: 0.04s
Presolved: 20137 rows, 3629 columns, 56321 nonzeros
Presolved model has 992 quadratic objective terms
Ordering time: 0.01s

Barrier statistics:
 Dense cols : 1
 Free vars  : 3
 AA' NZ     : 2.565e+05
 Factor NZ  : 2.858e+05 (roughly 12 MBytes of memory)
 Factor Ops : 5.951e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.48230649e+08 -2.52381543e+08  2.50e+03 1.00e+03  1.00e+06     0s
   1   2.31091513e+08

In [9]:
b

<gurobi.Var b (value 0.8180681711182086)>

In [10]:
a

<gurobi.Var a (value 0.18193182888179138)>